# Test name: qa_test_09

##### Reason for test:  
To test more complicated distributions where there are three supply points and demand points in a ring surrounding it - in this case the top two supplies can only support three demand points leaving the others to be redistributed to the final bottom suppliy which is large enough to take on the extra demand.

##### Parameters

Three supplies - vertically aligned - 150km apart

8 demands for each supply - in an even circle - a fixed distance away (50km)

Capacities - A - supply capable of sustaining 3 demand points, B - supply capable of sustaining 3 demand points, C - supply capable of sustaining 18 demand points.

##### Predicted outcome:
For the top supply (A) - the three top most demands should be attached to it. The two demands horizontally aligned with supply A plus the bottom left diagonal demand should all attach to supply B. All other demands should attach to supply C (2 from the demands surround A and all of the demands surrounding B and C).

This is based on the logic of minimising any single large distances in favour of having less of the shortest distances.


In [5]:
import pandas as pd
import psycopg2
import random
import os
from utilities import generate_point_dict 
from utilities import locate_supplies
from utilities import calculate_distances
dir = os.getcwd()
parent_directory = os.path.split(os.path.split(dir)[0])[0]

An example of what the csv data should look like (see column naming)

In [6]:
pd.read_csv(parent_directory + "/web/data/datasets/data.csv", nrows=2)

,demand,demand_id,demand_lat,demand_lng,demand_name,supply,supply_id,supply_lat,supply_lng,supply_name,duration_min,distance_crowflies_km,distance_route_km
0,11.662019,0,50.920128,-2.670739,Green Ln,35.689500,0,51.749314,-0.240863,Roehyde Way,158.883333,192.761855,235.458
1,11.662019,0,50.920128,-2.670739,Green Ln,28.249443,1,50.815128,-2.273901,Deer Park,48.433333,30.269113,50.560


The method for locating demands will change depending on the scenario and the locate_demands function may be different for each different test.

Changed to only give demand points for the third supply out of the three

In [7]:
def locate_demands(supply_df, num_demands_per_supply, dem_distances_km, dem_bearings_degrees, demand_vol, conn):
    demands = []
    supply_counter = 0
    
    supply_dict = supply_df.to_dict(orient="records")
    this_supply = {"lat":supply_dict[2]["supply_lat"], "lng": supply_dict[2]["supply_lng"]}
    for i in range(num_demands_per_supply):
        distance_km = dem_distances_km[i]
        bearing_degrees = dem_bearings_degrees[i]
        this_demand = generate_point_dict(this_supply, bearing_degrees, distance_km, conn)
        this_demand["demand"] = random.choice(demand_vol)
        this_demand["demand_name"] = "demand" + str(supply_counter + i)
        demands.append(this_demand)
    supply_counter = supply_counter + num_demands_per_supply

    demands_df = pd.DataFrame(demands).reset_index()
    demands_df.columns=["demand_id", "demand", "demand_name", "demand_lat", "demand_lng"]
    return(demands_df)

### Run from here to create a new test data set based on the parameters which can be changed below.

In [8]:
# Naming parameters
output_name = "qa_data_09_geo_proof"

# Supply parameters
num_supplies = 3
sup_distances_km = [0, 150, 300]
sup_bearings_degrees = [0, 0, 0]
supply_vol = [100, 100, 0]

# Demand parameters
num_demands_per_supply = 2
dem_distances_km = [50, 50]
dem_bearings_degrees = [90, 135] # Demands will be taken at these exact degrees for each supply
demand_vol = [100]

# Misc
scale = 1 # Used to scale demand to be a proportion of supply (1 by default)

In [9]:
conn = "host='localhost' dbname='postgres' user='postgres' password=''"
conn = psycopg2.connect(conn)

supply_df = locate_supplies(num_supplies, sup_distances_km, sup_bearings_degrees, supply_vol, conn)
demands_df = locate_demands(supply_df, num_demands_per_supply, dem_distances_km, dem_bearings_degrees, demand_vol, conn)


In [10]:
supply_df

,supply_id,supply_lat,supply_lng,supply,supply_name
0,0,51.501109,-1.242375,100,supply0
1,1,52.849173,-1.242375,100,supply1
2,2,54.196930,-1.242375,0,supply2


In [11]:
demands_df

,demand_id,demand,demand_name,demand_lat,demand_lng
0,0,100,demand0,54.194494,-0.476308
1,1,100,demand1,53.878081,-0.704790


In [10]:
# Scale supply capacity so that supply>demand by "scale" (ONLY RUN IF SCALING REQUIRED)
#supply_df["supply"] = supply_df["supply"]* demands_df["demand"].sum()/supply_df["supply"].sum()*scale

In [12]:
# Cartestian product
supply_df["cart"] = 1
demands_df["cart"] = 1
all_combos = demands_df.merge(supply_df)
all_combos = all_combos.drop("cart", axis=1)
all_combos

,demand_id,demand,demand_name,demand_lat,demand_lng,supply_id,supply_lat,supply_lng,supply,supply_name
0,0,100,demand0,54.194494,-0.476308,0,51.501109,-1.242375,100,supply0
1,0,100,demand0,54.194494,-0.476308,1,52.849173,-1.242375,100,supply1
2,0,100,demand0,54.194494,-0.476308,2,54.196930,-1.242375,0,supply2
3,1,100,demand1,53.878081,-0.704790,0,51.501109,-1.242375,100,supply0
4,1,100,demand1,53.878081,-0.704790,1,52.849173,-1.242375,100,supply1
5,1,100,demand1,53.878081,-0.704790,2,54.196930,-1.242375,0,supply2


In [13]:
# Calculate the crowflies distances for each combination
all_combos = calculate_distances(all_combos, conn)
all_combos.head(10)

,demand_id,demand,demand_name,demand_lat,demand_lng,supply_id,supply_lat,supply_lng,supply,supply_name,duration_min,distance_crowflies_km,distance_route_km
0,0,100,demand0,54.194494,-0.476308,0,51.501109,-1.242375,100,supply0,304.044672,304.044672,304.044672
1,0,100,demand0,54.194494,-0.476308,1,52.849173,-1.242375,100,supply1,158.065220,158.065220,158.065220
2,0,100,demand0,54.194494,-0.476308,2,54.196930,-1.242375,0,supply2,49.985008,49.985008,49.985008
3,1,100,demand1,53.878081,-0.704790,0,51.501109,-1.242375,100,supply0,266.911066,266.911066,266.911066
4,1,100,demand1,53.878081,-0.704790,1,52.849173,-1.242375,100,supply1,119.934061,119.934061,119.934061
5,1,100,demand1,53.878081,-0.704790,2,54.196930,-1.242375,0,supply2,49.984325,49.984325,49.984325


In [14]:
# Save file to CSV
path = dir + "/qa_data_files/" + str(output_name) + ".csv"
all_combos.to_csv(path, index=False)